In [2]:
!pip install requests

  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)


In [4]:
!pip install beautifulsoup4

In [6]:
!pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


In [69]:
!pip install pyecharts

  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13704 sha256=a73e71ead88b31476cf04f68b91fc2aea52dece9201b43bf5060a2f3fe9117c2
  Stored in directory: c:\users\shine\appdata\local\pip\cache\wheels\8c\76\0b\eb9eb3da7e2335e3577e3f96a0ae9f74f206e26457bd1a2bc8
Successfully built prettytable


In [1]:
# basic
import time
import random
import os
import re
import warnings
warnings.filterwarnings('ignore')

# data analytics
import numpy as np
import pandas as pd

# web crawling
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# scrapy
#import scrapy
#from scrapy.http import TextResponse # xpath 연습

# database
#import pymongo

# visulazation
#from pyvirtualdisplay import Display
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# chatbot
#from flask import *
#import konlpy
#from konlpy.tag import *

In [2]:
# matplotlib 한글 사용을 위한 코드
from matplotlib import rc
rc('font', family='AppleGothic')
mpl.rc('font', family='Hancom Gothic')
mpl.rc('axes', unicode_minus=False)

In [3]:
url = 'https://m.toomics.com/webtoon/top100/type/monthly'
#response = requests.get(url)
#dom = BeautifulSoup(response.text, 'html.parser')

### Top100: rank, title, genre, episodes, link

In [2]:
# try2 : selenium

from selenium import webdriver

In [46]:
driver = webdriver.Chrome()
driver.get(url)

In [49]:
top100 = driver.find_elements_by_css_selector('#contents > div:nth-child(4) > ul > li') 

In [50]:
len(top100)

100

In [51]:
# title
top100[0].find_element_by_css_selector('a > div > div > strong').text

'철보: [빌런]'

In [52]:
# order
top100[0].find_element_by_css_selector('a > div > span > span').text[:-1]

'1'

In [53]:
# count_episode & genre
epi_genre = top100[0].find_element_by_css_selector('a > div.toon__caption > span > span').text

In [54]:
# split count_episode
epi_genre.split("·")[0].replace(" ", "")

'95화'

In [55]:
# link 
link = top100[0].find_element_by_css_selector('a').get_attribute('href')

In [56]:
# split cartoon id
link.split('/')[-1]

'5046'

In [57]:
# splite genre
epi_genre.split("·")[1].replace(" ", "")

'학원/액션'

In [64]:
ranks = []
titles = []
epi_genre_list = []
links = []

for i in range(len(top100)):
    rank = top100[i].find_element_by_css_selector('a > div > span > span').text[:-1]
    title = top100[i].find_element_by_css_selector('a > div > div > strong').text
    epi_genre = top100[i].find_element_by_css_selector('a > div.toon__caption > span > span').text
    link = top100[i].find_element_by_css_selector('a').get_attribute('href')
    
    ranks.append(rank)
    titles.append(title)
    epi_genre_list.append(epi_genre)
    links.append(link)

In [65]:
len(ranks), len(titles), len(epi_genre_list), len(links) 

(100, 100, 100, 100)

In [67]:
#ranks

In [68]:
Toomics_Top100_monthly = {
    'rank' : ranks,
    'title' : titles,
    'epi_genre_list' : epi_genre_list,
    'link' : links
}

In [77]:
driver.quit()

In [110]:
#Toomics_Top100_monthly

In [69]:
df_Toomics_Top100_monthly = pd.DataFrame(Toomics_Top100_monthly)
df_Toomics_Top100_monthly

,rank,title,epi_genre_list,link
0,1,철보: [빌런],95화 · 학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...
1,2,소년의 신성,142화 · 학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...
2,3,로스타임 인 유럽,184화 · 스포츠,https://m.toomics.com/webtoon/bridge/type/2/to...
3,4,고교정점,260화 · 학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...
4,5,고교전설,197화 · 학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...
...,...,...,...,...
95,96,나의 위험한 수호신,100화 · 로맨스,https://m.toomics.com/webtoon/bridge/type/2/to...
96,97,엉덩국 만화공장,70화 · 개그,https://m.toomics.com/webtoon/bridge/type/2/to...
97,98,다이다이,33화 · 학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...
98,99,블러드 블레이드,36화 · 판타지,https://m.toomics.com/webtoon/bridge/type/2/to...


In [70]:
# loop split count_episode, genre
df_Toomics_Top100_monthly['count_episode'] = ''
df_Toomics_Top100_monthly['genre'] = ''

for i in range(len(df_Toomics_Top100_monthly)):
    df_Toomics_Top100_monthly['count_episode'][i] = df_Toomics_Top100_monthly['epi_genre_list'][i].split("·")[0].replace(" ", "")
    df_Toomics_Top100_monthly['genre'][i] = df_Toomics_Top100_monthly['epi_genre_list'][i].split("·")[1].replace(" ", "")


In [71]:
# loop extract webtoon id from link
df_Toomics_Top100_monthly['id'] = ''
for i in range(len(df_Toomics_Top100_monthly)):
    df_Toomics_Top100_monthly['id'][i] = df_Toomics_Top100_monthly['link'][i].split('/')[-1]

In [72]:
df_Toomics_Top100_monthly

,rank,title,epi_genre_list,link,count_episode,genre,id
0,1,철보: [빌런],95화 · 학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,95화,학원/액션,5046
1,2,소년의 신성,142화 · 학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,142화,학원/액션,4626
2,3,로스타임 인 유럽,184화 · 스포츠,https://m.toomics.com/webtoon/bridge/type/2/to...,184화,스포츠,2371
3,4,고교정점,260화 · 학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,260화,학원/액션,1695
4,5,고교전설,197화 · 학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,197화,학원/액션,4538
...,...,...,...,...,...,...,...
95,96,나의 위험한 수호신,100화 · 로맨스,https://m.toomics.com/webtoon/bridge/type/2/to...,100화,로맨스,5082
96,97,엉덩국 만화공장,70화 · 개그,https://m.toomics.com/webtoon/bridge/type/2/to...,70화,개그,4686
97,98,다이다이,33화 · 학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,33화,학원/액션,4878
98,99,블러드 블레이드,36화 · 판타지,https://m.toomics.com/webtoon/bridge/type/2/to...,36화,판타지,4726


In [73]:
df_Toomics100 = df_Toomics_Top100_monthly[['rank', 'id', 'title', 'count_episode', 'genre', 'link']]

In [74]:
df_Toomics100.to_csv("./df_Toomics100.csv", index = False)

### detail information

In [3]:
df_Toomics100 = pd.read_csv("./df_Toomics100.csv").reset_index(drop=True)

In [4]:
df_Toomics100

,rank,id,title,count_episode,genre,link
0,1,5046,철보: [빌런],95화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...
1,2,4626,소년의 신성,142화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...
2,3,2371,로스타임 인 유럽,184화,스포츠,https://m.toomics.com/webtoon/bridge/type/2/to...
3,4,1695,고교정점,260화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...
4,5,4538,고교전설,197화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...
...,...,...,...,...,...,...
95,96,5082,나의 위험한 수호신,100화,로맨스,https://m.toomics.com/webtoon/bridge/type/2/to...
96,97,4686,엉덩국 만화공장,70화,개그,https://m.toomics.com/webtoon/bridge/type/2/to...
97,98,4878,다이다이,33화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...
98,99,4726,블러드 블레이드,36화,판타지,https://m.toomics.com/webtoon/bridge/type/2/to...


In [5]:
#get detail link

#format : https://m.toomics.com/webtoon/episode/toon/<id>

df_Toomics100['full_link'] = ''

for i in range(len(df_Toomics100)):
    df_Toomics100['full_link'][i] = "https://m.toomics.com/webtoon/episode/toon/" + str(df_Toomics100['id'][i])


In [6]:
df_Toomics100

,rank,id,title,count_episode,genre,link,full_link
0,1,5046,철보: [빌런],95화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/5046
1,2,4626,소년의 신성,142화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/4626
2,3,2371,로스타임 인 유럽,184화,스포츠,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/2371
3,4,1695,고교정점,260화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/1695
4,5,4538,고교전설,197화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/4538
...,...,...,...,...,...,...,...
95,96,5082,나의 위험한 수호신,100화,로맨스,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/5082
96,97,4686,엉덩국 만화공장,70화,개그,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/4686
97,98,4878,다이다이,33화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/4878
98,99,4726,블러드 블레이드,36화,판타지,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/4726


In [6]:
detail_link = df_Toomics100['full_link'][0]
detail_link

'https://m.toomics.com/webtoon/episode/toon/5046'

In [14]:
df_Toomics100['keywords'] = ''
df_Toomics100['each_published_dates'] = ''

In [18]:
for i in range(len(df_Toomics100)):
    
    
    detail_link = df_Toomics100['full_link'][i]
    
    driver = webdriver.Chrome()
    driver.get(detail_link)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    
    # crawling_keywords
    keywords = driver.find_elements_by_css_selector('#contents >  div > div.episode__header > span.epsidoe__tags > a')
    print(len(keywords))
    
   
    each_cartoon_keywords = []

    for n in range(len(keywords)):
        keyword = keywords[n].text
        each_cartoon_keywords.append(keyword)
    df_Toomics100['keywords'][i] = each_cartoon_keywords
    print('complete_cartoon_keyword', df_Toomics100['keywords'][:i])
    
    # episode_published_date
    episode_list = []
    episodes = driver.find_element_by_css_selector('#contents > div > div.episode__body > ul.eps').text
    episode_list = episodes.split('제')[1:]
    print(len(episode_list))
    
    each_episode_published_dates = []
    for s in range(len(episode_list)):
        
        each_episode_published_date = episode_list[s].split('\n')
        print(each_episode_published_date)
        each_episode_published_dates.append(each_episode_published_date)
    df_Toomics100['each_published_dates'][i] = each_episode_published_dates
    print('complite_catoon_date')
    
    driver.quit()
    

6
complete_cartoon_keyword Series([], Name: keywords, dtype: object)
95
['1화', '철보: [villain]', '2018.12.23', '무료', '']
['2화', '철보: [villain]', '2018.12.23', '무료', '']
['3화', '철보: [villain]', '2018.12.23', '무료', '']
['4화', '철보: [villain]', '2018.12.23', '무료', '']
['5화', '철보: [villain]', '2018.12.23', '무료', '']
['6화', '철보: [villain]', '2018.12.23', '무료', '']
['7화', '철보: [villain]', '2018.12.23', '로그인', '필요', '']
['8화', '철보: [villain]', '2018.12.23', '로그인', '필요', '']
['9화', '철보: [villain]', '2018.12.23', '로그인', '필요', '']
['10화', '철보: [villain]', '2018.12.23', '로그인', '필요', '']
['11화', '철보: [villain]', '2018.12.23', '로그인', '필요', '']
['12화', '철보: [villain]', '2018.12.23', '로그인', '필요', '']
['13화', '철보: [villain]', '2018.12.23', '로그인', '필요', '']
['14화', '철보: [villain]', '2018.12.23', '로그인', '필요', '']
['15화', '철보: [villain]', '2018.12.30', '로그인', '필요', '']
['16화', '철보: [villain]', '2019.01.06', '로그인', '필요', '']
['17화', '철보: [villain]', '2019.01.13', '로그인', '필요', '']
['18화', '철보: [villain]', '2

4
complete_cartoon_keyword 0          [#일요연재, #필독웹툰, #일진, #고딩, #의리, #남자들의세계]
1    [#월요연재, #필독웹툰, #남자들의세계, #고딩, #일진, #히어로, #주먹]
Name: keywords, dtype: object
184
['1화', '로스타임 인 유럽', '2016.04.16', '무료', '']
['2화', '로스타임 인 유럽', '2016.04.16', '무료', '']
['3화', '로스타임 인 유럽', '2016.04.23', '무료', '']
['4화', '로스타임 인 유럽', '2016.04.23', '무료', '']
['5화', '로스타임 인 유럽', '2016.04.30', '무료', '']
['6화', '로스타임 인 유럽', '2016.05.07', '무료', '']
['7화', '로스타임 인 유럽', '2016.05.14', '무료', '']
['8화', '로스타임 인 유럽', '2016.05.21', '무료', '']
['9화', '로스타임 인 유럽', '2016.05.28', '무료', '']
['10화', '로스타임 인 유럽', '2016.06.04', '로그인', '필요', '']
['11화', '로스타임 인 유럽', '2016.07.09', '로그인', '필요', '']
['12화', '로스타임 인 유럽', '2016.07.16', '로그인', '필요', '']
['13화', '로스타임 인 유럽', '2016.07.23', '로그인', '필요', '']
['14화', '로스타임 인 유럽', '2016.07.30', '로그인', '필요', '']
['15화', '로스타임 인 유럽', '2016.08.06', '로그인', '필요', '']
['16화', '로스타임 인 유럽', '2016.08.13', '로그인', '필요', '']
['17화', '로스타임 인 유럽', '2016.08.20', '로그인', '필요', '']
['18화', '로스타임 인 유럽', '2016.

7
complete_cartoon_keyword 0          [#일요연재, #필독웹툰, #일진, #고딩, #의리, #남자들의세계]
1    [#월요연재, #필독웹툰, #남자들의세계, #고딩, #일진, #히어로, #주먹]
2                      [#완결, #필독웹툰, #축구, #남자들의세계]
Name: keywords, dtype: object
266
['1화', '학교', '2015.08.14', '무료', '']
['2화', '새로운 삶', '2015.08.14', '무료', '']
['3화', '갈등의 시작', '2015.08.14', '무료', '']
['4화', '압도', '2015.08.21', '무료', '']
['5화', '선배', '2015.08.28', '무료', '']
['6화', '전국구', '2015.09.04', '무료', '']
['7화', '태산회', '2015.09.11', '무료', '']
['8화', '고민', '2015.09.18', '무료', '']
['9화', '시험', '2015.09.25', '무료', '']
['10화', '여러 갈등(상)', '2015.10.02', '무료', '']
['11화', '여러 갈등(하)', '2015.10.09', '로그인', '필요', '']
['12화', '바닥', '2015.10.16', '로그인', '필요', '']
['13화', '팔극권', '2015.10.23', '로그인', '필요', '']
['14화', '전쟁', '2015.10.30', '로그인', '필요', '']
['15화', '북남고교', '2015.11.06', '로그인', '필요', '']
['16화', '전쟁의 시작', '2015.11.13', '로그인', '필요', '']
['17화', '패싸움', '2015.11.20', '로그인', '필요', '']
['18화', '단련', '2015.11.27', '로그인', '필요', '']
['19화', '이즈미', '2015.12.04', 

7
complete_cartoon_keyword 0          [#일요연재, #필독웹툰, #일진, #고딩, #의리, #남자들의세계]
1    [#월요연재, #필독웹툰, #남자들의세계, #고딩, #일진, #히어로, #주먹]
2                      [#완결, #필독웹툰, #축구, #남자들의세계]
3     [#금요연재, #필독웹툰, #고딩, #일진, #남자들의세계, #친구, #우정]
Name: keywords, dtype: object
198
['1화', '우리형', '2016.10.29', '무료', '']
['2화', '세력 다툼', '2016.10.29', '무료', '']
['3화', '서클 모집', '2016.10.29', '무료', '']
['4화', '사건의 시작', '2016.10.29', '무료', '']
['5화', '사고뭉치', '2016.10.29', '무료', '']
['6화', '위기', '2016.11.05', '무료', '']
['7화', '강화상고 박태진', '2016.11.12', '로그인', '필요', '']
['8화', '강민준 vs 대머리', '2016.11.19', '로그인', '필요', '']
['9화', '난입꾼', '2016.11.26', '로그인', '필요', '']
['10화', '난입꾼 (2)', '2016.12.03', '로그인', '필요', '']
['11화', '오해와 오해', '2016.12.10', '로그인', '필요', '']
['12화', '학교 생활', '2016.12.17', '로그인', '필요', '특별판', '고교정점&고교전설 특별편_세계관 정리', '2016.12.24', '무료', '']
['13화', '여유', '2016.12.24', '로그인', '필요', '']
['14화', '자질', '2016.12.31', '로그인', '필요', '']
['15화', '최강 신입생', '2017.01.07', '로그인', '필요', '']
['16화', '사건의 연속', '2

9
complete_cartoon_keyword 0          [#일요연재, #필독웹툰, #일진, #고딩, #의리, #남자들의세계]
1    [#월요연재, #필독웹툰, #남자들의세계, #고딩, #일진, #히어로, #주먹]
2                      [#완결, #필독웹툰, #축구, #남자들의세계]
3     [#금요연재, #필독웹툰, #고딩, #일진, #남자들의세계, #친구, #우정]
4     [#토요연재, #필독웹툰, #남자들의세계, #고딩, #일진, #친구, #우정]
Name: keywords, dtype: object
69
['1화 심해수', '어항속의 물고기 (上)', '2018.03.19', '무료', '']
['2화 심해수', '어항속의 물고기 (下)', '2018.03.19', '무료', '']
['3화 심해수', '습격 (上)', '2018.03.19', '무료', '']
['4화 심해수', '습격 (下)', '2018.03.19', '로그인', '필요', '']
['5화 심해수', '리타 그리고 나 (上)', '2018.03.19', '로그인', '필요', '']
['6화 심해수', '리타 그리고 나 (下)', '2018.03.19', '로그인', '필요', '']
['7화 심해수', '지평선 너머 (上)', '2018.03.19', '로그인', '필요', '']
['8화 심해수', '지평선 너머 (下)', '2018.03.19', '로그인', '필요', '']
['9화 심해수', '작살꾼 (上)', '2018.03.19', '로그인', '필요', '']
['10화 심해수', '작살꾼 (下)', '2018.03.19', '로그인', '필요', '']
['11화 심해수', '유니온 부산 (上)', '2018.04.16', '로그인', '필요', '']
['12화 심해수', '유니온 부산 (下)', '2018.04.16', '로그인', '필요', '']
['13화 심해수', '난민선단 (上)', '2018.05.21', '로그인

...

4
complete_cartoon_keyword 0           [#일요연재, #필독웹툰, #일진, #고딩, #의리, #남자들의세계]
1     [#월요연재, #필독웹툰, #남자들의세계, #고딩, #일진, #히어로, #주먹]
2                       [#완결, #필독웹툰, #축구, #남자들의세계]
3      [#금요연재, #필독웹툰, #고딩, #일진, #남자들의세계, #친구, #우정]
4      [#토요연재, #필독웹툰, #남자들의세계, #고딩, #일진, #친구, #우정]
                          ...                     
82                    [#완결, #영화예정, #능력자, #사주, #운명]
83                         [#완결, #15금, #초능력, #먼치킨]
84                                 [#완결, #전쟁, #사랑]
85                             [#완결, #남자들의세계, #조폭]
86                         [#금요연재, #투윅스, #마족, #악당]
Name: keywords, Length: 87, dtype: object
29
['1화', '폭풍의 전학생', '2020.02.19', '무료', '']
['2화', '세기의 대결', '2020.02.19', '무료', '']
['3화', '청룡회의 탄생', '2020.02.19', '무료', '']
['4화', '김훈의 실력', '2020.02.19', '무료', '']
['5화', '타도 김훈 전략', '2020.02.19', '무료', '']
['6화', '학교 짱', '2020.02.19', '무료', '']
['7화', '완전체 동아리', '2020.02.26', '무료', '']
['8화', '태권고등학교의 위기 (1)', '2020.03.04', '2코인', '무료', '']
['9화', '태권고등학교의 위기 (2

4
complete_cartoon_keyword 0           [#일요연재, #필독웹툰, #일진, #고딩, #의리, #남자들의세계]
1     [#월요연재, #필독웹툰, #남자들의세계, #고딩, #일진, #히어로, #주먹]
2                       [#완결, #필독웹툰, #축구, #남자들의세계]
3      [#금요연재, #필독웹툰, #고딩, #일진, #남자들의세계, #친구, #우정]
4      [#토요연재, #필독웹툰, #남자들의세계, #고딩, #일진, #친구, #우정]
                          ...                     
86                         [#금요연재, #투윅스, #마족, #악당]
87                         [#수요연재, #코믹, #고딩, #히어로]
88            [#완결, #통시리즈, #남자들의세계, #일진, #고딩, #의리]
89          [#완결, #월간투믹스, #남자들의세계, #고딩, #찌질남, #일진]
90                            [#완결, #고딩, #비밀, #코믹]
Name: keywords, Length: 91, dtype: object
70
['1화', '망태', '2020.05.26', '무료', '']
['2화', '망태', '2020.05.26', '무료', '']
['3화', '망태', '2020.05.26', '무료', '']
['4화', '망태', '2020.05.26', '무료', '']
['5화', '망태', '2020.05.26', '무료', '']
['6화', '망태', '2020.05.26', '로그인', '필요', '']
['7화', '망태', '2020.05.26', '로그인', '필요', '']
['8화', '망태', '2020.05.26', '로그인', '필요', '']
['9화', '망태', '2020.05.26', '로그인', '필요', '']
['10화'

3
complete_cartoon_keyword 0           [#일요연재, #필독웹툰, #일진, #고딩, #의리, #남자들의세계]
1     [#월요연재, #필독웹툰, #남자들의세계, #고딩, #일진, #히어로, #주먹]
2                       [#완결, #필독웹툰, #축구, #남자들의세계]
3      [#금요연재, #필독웹툰, #고딩, #일진, #남자들의세계, #친구, #우정]
4      [#토요연재, #필독웹툰, #남자들의세계, #고딩, #일진, #친구, #우정]
                          ...                     
88            [#완결, #통시리즈, #남자들의세계, #일진, #고딩, #의리]
89          [#완결, #월간투믹스, #남자들의세계, #고딩, #찌질남, #일진]
90                            [#완결, #고딩, #비밀, #코믹]
91                            [#완결, #요괴, #소년, #영웅]
92                                 [#완결, #전쟁, #악마]
Name: keywords, Length: 93, dtype: object
77
['1화', '환생', '2017.05.31', '무료', '']
['2화', '고백', '2017.05.31', '무료', '']
['3화', '연결고리', '2017.05.31', '로그인', '필요', '']
['4화', '언젠가 설명이 필요한 밤', '2017.05.31', '로그인', '필요', '']
['5화', '습관', '2017.05.31', '로그인', '필요', '']
['6화', '어느 별에서 왔니', '2017.05.31', '로그인', '필요', '']
['7화', '썸', '2017.05.31', '로그인', '필요', '']
['8화', '시간을 달려서', '2017.05.31', '로그인', '필요', '']
['9화'

4
complete_cartoon_keyword 0           [#일요연재, #필독웹툰, #일진, #고딩, #의리, #남자들의세계]
1     [#월요연재, #필독웹툰, #남자들의세계, #고딩, #일진, #히어로, #주먹]
2                       [#완결, #필독웹툰, #축구, #남자들의세계]
3      [#금요연재, #필독웹툰, #고딩, #일진, #남자들의세계, #친구, #우정]
4      [#토요연재, #필독웹툰, #남자들의세계, #고딩, #일진, #친구, #우정]
                          ...                     
91                            [#완결, #요괴, #소년, #영웅]
92                                 [#완결, #전쟁, #악마]
93                                [#완결, #15금, #고딩]
94                      [#완결, #캠퍼스, #찌질남, #키보드워리어]
95               [#완결, #미소녀, #미소년, #마법, #수호신, #환생]
Name: keywords, Length: 96, dtype: object
72
['1화', '엉덩국 만화공장의 위기', '2017.04.21', '무료', '']
['2화', '공포의 외국인', '2017.04.21', '무료', '']
['3화', '0.1%의 확률', '2017.04.21', '무료', '']
['4화', '아기에게도 피지컬이 있다', '2017.04.21', '무료', '']
['5화', '대출은 신중하게!', '2017.04.21', '무료', '']
['6화', '곰팡이의 인생', '2017.04.21', '무료', '']
['7화', '행복한 결혼식', '2017.04.21', '무료', '']
['8화', '입이 무거운 소년', '2017.04.21', '무료', '']
['9화', '우리 반 부반

6
complete_cartoon_keyword 0           [#일요연재, #필독웹툰, #일진, #고딩, #의리, #남자들의세계]
1     [#월요연재, #필독웹툰, #남자들의세계, #고딩, #일진, #히어로, #주먹]
2                       [#완결, #필독웹툰, #축구, #남자들의세계]
3      [#금요연재, #필독웹툰, #고딩, #일진, #남자들의세계, #친구, #우정]
4      [#토요연재, #필독웹툰, #남자들의세계, #고딩, #일진, #친구, #우정]
                          ...                     
94                      [#완결, #캠퍼스, #찌질남, #키보드워리어]
95               [#완결, #미소녀, #미소년, #마법, #수호신, #환생]
96                            [#완결, #병맛, #코믹, #막장]
97                   [#완결, #월간투믹스, #찌질남, #가족, #복수]
98                [#완결, #월간투믹스, #뱀파이어, #인류종말, #전쟁]
Name: keywords, Length: 99, dtype: object
45
['1화', '마이 코튼 캔디', '2018.02.04', '무료', '']
['2화', '마이 코튼 캔디', '2018.02.04', '무료', '']
['3화', '마이 코튼 캔디', '2018.02.04', '무료', '']
['4화', '마이 코튼 캔디', '2018.02.04', '무료', '']
['5화', '마이 코튼 캔디', '2018.02.04', '무료', '']
['6화', '마이 코튼 캔디', '2018.02.04', '무료', '']
['7화', '마이 코튼 캔디', '2018.02.04', '무료', '']
['8화', '마이 코튼 캔디', '2018.02.04', '무료', '']
['9화', '마이 코튼 캔디', '201

In [15]:
driver.quit()

In [20]:
df_Toomics100

,rank,id,title,count_episode,genre,link,full_link,keywords,each_published_dates
0,1,5046,철보: [빌런],95화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/5046,"[#일요연재, #필독웹툰, #일진, #고딩, #의리, #남자들의세계]","[[1화, 철보: [villain], 2018.12.23, 무료, ], [2화, 철..."
1,2,4626,소년의 신성,142화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/4626,"[#월요연재, #필독웹툰, #남자들의세계, #고딩, #일진, #히어로, #주먹]","[[1화, 소년의 신성, 2016.12.26, 무료, ], [2화, 소년의 신성, ..."
2,3,2371,로스타임 인 유럽,184화,스포츠,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/2371,"[#완결, #필독웹툰, #축구, #남자들의세계]","[[1화, 로스타임 인 유럽, 2016.04.16, 무료, ], [2화, 로스타임 ..."
3,4,1695,고교정점,260화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/1695,"[#금요연재, #필독웹툰, #고딩, #일진, #남자들의세계, #친구, #우정]","[[1화, 학교, 2015.08.14, 무료, ], [2화, 새로운 삶, 2015...."
4,5,4538,고교전설,197화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/4538,"[#토요연재, #필독웹툰, #남자들의세계, #고딩, #일진, #친구, #우정]","[[1화, 우리형, 2016.10.29, 무료, ], [2화, 세력 다툼, 2016..."
...,...,...,...,...,...,...,...,...,...
95,96,5082,나의 위험한 수호신,100화,로맨스,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/5082,"[#완결, #미소녀, #미소년, #마법, #수호신, #환생]","[[1화, 나의 위험한 수호신 시즌1, 2019.02.26, 무료, ], [2화, ..."
96,97,4686,엉덩국 만화공장,70화,개그,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/4686,"[#완결, #병맛, #코믹, #막장]","[[1화, 엉덩국 만화공장의 위기, 2017.04.21, 무료, ], [2화, 공포..."
97,98,4878,다이다이,33화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/4878,"[#완결, #월간투믹스, #찌질남, #가족, #복수]","[[1화, 다이다이, 2018.03.19, 무료, ], [2화, 다이다이, 2018..."
98,99,4726,블러드 블레이드,36화,판타지,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/4726,"[#완결, #월간투믹스, #뱀파이어, #인류종말, #전쟁]","[[1화, 유령 PART_01, 2018.03.12, 무료, ], [2화, 유령 P..."


In [21]:
df_Toomics100.to_csv("./df_Toomics100_2.csv", index = False)

In [22]:
#df_Toomics100['each_published_dates'][0]

In [1]:
pd.read_csv("./df_Toomics100_2.csv")

,rank,id,title,count_episode,genre,link,full_link,keywords,each_published_dates
0,1,5046,철보: [빌런],95화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/5046,"['#일요연재', '#필독웹툰', '#일진', '#고딩', '#의리', '#남자들의...","[['1화', '철보: [villain]', '2018.12.23', '무료', '..."
1,2,4626,소년의 신성,142화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/4626,"['#월요연재', '#필독웹툰', '#남자들의세계', '#고딩', '#일진', '#...","[['1화', '소년의 신성', '2016.12.26', '무료', ''], ['2..."
2,3,2371,로스타임 인 유럽,184화,스포츠,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/2371,"['#완결', '#필독웹툰', '#축구', '#남자들의세계']","[['1화', '로스타임 인 유럽', '2016.04.16', '무료', ''], ..."
3,4,1695,고교정점,260화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/1695,"['#금요연재', '#필독웹툰', '#고딩', '#일진', '#남자들의세계', '#...","[['1화', '학교', '2015.08.14', '무료', ''], ['2화', ..."
4,5,4538,고교전설,197화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/4538,"['#토요연재', '#필독웹툰', '#남자들의세계', '#고딩', '#일진', '#...","[['1화', '우리형', '2016.10.29', '무료', ''], ['2화',..."
...,...,...,...,...,...,...,...,...,...
95,96,5082,나의 위험한 수호신,100화,로맨스,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/5082,"['#완결', '#미소녀', '#미소년', '#마법', '#수호신', '#환생']","[['1화', '나의 위험한 수호신 시즌1', '2019.02.26', '무료', ..."
96,97,4686,엉덩국 만화공장,70화,개그,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/4686,"['#완결', '#병맛', '#코믹', '#막장']","[['1화', '엉덩국 만화공장의 위기', '2017.04.21', '무료', ''..."
97,98,4878,다이다이,33화,학원/액션,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/4878,"['#완결', '#월간투믹스', '#찌질남', '#가족', '#복수']","[['1화', '다이다이', '2018.03.19', '무료', ''], ['2화'..."
98,99,4726,블러드 블레이드,36화,판타지,https://m.toomics.com/webtoon/bridge/type/2/to...,https://m.toomics.com/webtoon/episode/toon/4726,"['#완결', '#월간투믹스', '#뱀파이어', '#인류종말', '#전쟁']","[['1화', '유령 PART_01', '2018.03.12', '무료', ''],..."


In [125]:
rank = []

for i in range(1, 101):
    rank.append(i)
df_Toomics100['rank'] = rank

In [126]:
df_Toomics100 = df_Toomics100[['rank', 'title', 'count_episode', 'genre']]

In [127]:
df_Toomics100['genre'].unique()

array(['학원/액션', '스포츠', '판타지', '드라마', '개그', '무협/시대극', '로맨스', '공포/스릴러',
       'BL'], dtype=object)

In [128]:
df_Toomics100

,rank,title,count_episode,genre
0,1,철보: [빌런],95화,학원/액션
1,2,소년의 신성,142화,학원/액션
2,3,로스타임 인 유럽,184화,스포츠
3,4,고교정점,259화,학원/액션
4,5,고교전설,197화,학원/액션
...,...,...,...,...
95,96,사내자식,38화,드라마
96,97,엉덩국 만화공장,70화,개그
97,98,기인록,157화,학원/액션
98,99,다이다이,33화,학원/액션
